In [15]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout 

In [16]:
# Tasvirlarni yuklash va preprocess qilish
data_gen = ImageDataGenerator(
    rescale=1./255, 
    validation_split=0.2,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

train_generator = data_gen.flow_from_directory(
    'F:/Data Science/Computer Vision loyixa/Svetafor rangini aniqlovchi model/With_Deep_learning/dataset',
    target_size=(128, 128),  # Kattaroq tasvir o'lchami
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

validation_generator = data_gen.flow_from_directory(
    'F:/Data Science/Computer Vision loyixa/Svetafor rangini aniqlovchi model/With_Deep_learning/dataset',
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

# Modelni yaratish
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),  # Overfittingni oldini olish uchun
    Dense(128, activation='relu'),
    Dense(3, activation='softmax')  # Uchta sinf: Red, Green, Yellow
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Modelni o'qitish
model.fit(train_generator, validation_data=validation_generator, epochs=20)

# O'qitilgan modelni saqlash
model.save('traffic_light_model_advanced.h5')



Found 51 images belonging to 3 classes.
Found 10 images belonging to 3 classes.


c:\Users\Asus\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 16s 2s/step - accuracy: 0.4596 - loss: 1.1007 - val_accuracy: 0.3000 - val_loss: 1.0799
Epoch 2/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 7s 941ms/step - accuracy: 0.3760 - loss: 1.0984 - val_accuracy: 0.5000 - val_loss: 1.0963
Epoch 3/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 7s 959ms/step - accuracy: 0.5765 - loss: 0.9998 - val_accuracy: 0.6000 - val_loss: 0.9796
Epoch 4/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 6s 995ms/step - accuracy: 0.5893 - loss: 1.0271 - val_accuracy: 0.8000 - val_loss: 0.8951
Epoch 5/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 6s 744ms/step - accuracy: 0.7310 - loss: 0.9234 - val_accuracy: 0.7000 - val_loss: 0.8097
Epoch 6/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 6s 905ms/step - accuracy: 0.6725 - loss: 0.8400 - val_accuracy: 0.7000 - val_loss: 0.7595
Epoch 7/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 6s 733ms/step - accuracy: 0.6266 - loss: 0.8671 - val_accuracy: 0.7000 - val_loss: 0.7167
Epoch 8/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 6s 920ms/step - accuracy: 0.7031 - loss: 0.7854 - val_accuracy: 0.8000 - val_loss: 0

In [17]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model

# O'qitilgan modelni yuklash
model = load_model('traffic_light_model_advanced.h5')

# Tasvirni preprocess qilish uchun yordamchi funktsiya
def preprocess_image(img):
    img = cv2.resize(img, (128, 128))  # Kattaroq o'lcham
    img = img.astype('float32') / 255.0
    img = np.expand_dims(img, axis=0)
    return img




In [18]:
# Kamerani ochish
cap = cv2.VideoCapture(0)

while True:
    # Freymni olish
    ret, frame = cap.read()
    if not ret:
        break

    # Tasvirni preprocess qilish
    preprocessed_img = preprocess_image(frame)

    # Rangni aniqlash
    prediction = model.predict(preprocessed_img)
    class_idx = np.argmax(prediction)
    color = ["Red", "Green", "Yellow"][class_idx]

    # Rangni ko'rsatish
    cv2.putText(frame, f'Traffic Light Color: {color}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

    # Tasvirni ko'rsatish
    cv2.imshow('Traffic Light Detection', frame)

    # 'q' tugmasi bosilganda dasturni to'xtatish
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Kamerani yopish
cap.release()
cv2.destroyAllWindows()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 340ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
1/1 ━━━━━━━